# SLC4A1 Bruce et al., 1997. Distal renal tubular acidosis 1 	179800 

Data derived from [Bruce LJ (1997) Familial distal renal tubular acidosis is associated with mutations in the red cell anion exchanger (Band 3, AE1) gene. J Clin Invest 100:1693-707. PMID: 9312167](https://pubmed.ncbi.nlm.nih.gov/9312167/)

Variants were encoded as follows

-  family A: G->A mutation at 1766 (R589H) - NM_000342.4(SLC4A1):c.1766G>A (p.Arg589His)
-  family B: G->A mutation at 1766 (R589H) - NM_000342.4(SLC4A1):c.1766G>A (p.Arg589His)
-  family C: C->T mutation at 1765 (R589C) - NM_000342.4(SLC4A1):c.1765C>T (p.Arg589Cys)
-  family D: C->T mutation at 1838 (S613F) - NM_000342.4(SLC4A1):c.1838C>T (p.Ser613Phe) 

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import PhenopacketTable, QcVisualizer
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.34


In [2]:
PMID="PMID:9312167"
title = "Familial distal renal tubular acidosis is associated with mutations in the red cell anion exchanger (Band 3, AE1) gene"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-01-16


In [3]:
df = pd.read_csv("input/bruce_SLC4A1.tsv", sep='\t')

In [4]:
df.head(2)

,individual,Sex/Age,Creatinine(plasma),HCO3(plasma),K(Plasma),min_pH(urine),maximumOsm(urine),Age,nephrocalcinosis,kidney stones,dRTA,raised hemoglobin,raised hematocrti,NM_000342.4
0,A:I:1,F/60,74,24,4.1,6.26,635,74,No,Yes,Yes,No,No,c.1766G>A
1,A:I:2,F/40,86,18,3.4,6.35,572,71,Yes,Yes,No,No,No,c.1766G>A


In [5]:
generator = SimpleColumnMapperGenerator(df=df, hpo_cr=hpo_cr, observed="Yes", excluded="No")
column_mapper_list = generator.try_mapping_columns()
display(HTML(generator.to_html()))

Result,Columns
Mapped,nephrocalcinosis; kidney stones
Unmapped,individual; Sex/Age; Creatinine(plasma); HCO3(plasma); K(Plasma); min_pH(urine); maximumOsm(urine); Age; dRTA; raised hemoglobin; raised hematocrti; NM_000342.4


In [6]:
# Creatinine(plasma). Normal 50–125 mmol/liter
# Elevated circulating creatinine concentration HP:0003259
crea = Thresholder.creatinine_blood(unit="micromole/L", low_thresh=50, high_thresh=125)
creaTM = ThresholdedColumnMapper(column_name="Creatinine(plasma)", thresholder=crea)
column_mapper_list.append(creaTM)
creaTM.preview_column(df)

,mapping: 50.0-125.0 micromole/L,count
0,Abnormal circulating creatinine concentration (HP:0012100): excluded,16
1,Elevated circulating creatinine concentration (HP:0003259): observed,2


In [7]:
# HCO3(plasma) 22–31 
bicarb_low = HpTerm(hpo_id="HP:0032066", label="Decreased serum bicarbonate concentration")
bicarb = Thresholder(unit="mmol/liter", threshold_low=22, threshold_high=31, hpo_term_low=bicarb_low, hpo_term_abn=bicarb_low)
bicarbTM = ThresholdedColumnMapper(column_name="HCO3(plasma)", thresholder=bicarb)
column_mapper_list.append(bicarbTM)
bicarbTM.preview_column(df)

,mapping: 22.0-31.0 mmol/liter,count
0,Decreased serum bicarbonate concentration (HP:0032066): excluded,7
1,Decreased serum bicarbonate concentration (HP:0032066): observed,11


In [8]:
potassium = Thresholder.potassium_blood()
kTM = ThresholdedColumnMapper(column_name="K(Plasma)", thresholder=potassium)
column_mapper_list.append( kTM)
kTM.preview_column(df)

,mapping: 3.5-5.2 mEq/L,count
0,Abnormal blood potassium concentration (HP:0011042): excluded,12
1,Hypokalemia (HP:0002900): observed,6


In [9]:
# min_pH(urine) < 5.30
alku = HpTerm(hpo_id="HP:0032944", label="Alkaline urine")
alkUrine = Thresholder(unit="pH", threshold_high=5.30, hpo_term_high=alku, hpo_term_abn=alku)
alkU_TM = ThresholdedColumnMapper(column_name="min_pH(urine)", thresholder=alkUrine)
column_mapper_list.append(alkU_TM)
alkU_TM.preview_column(df)

,mapping: None-5.3 pH,count
0,Alkaline urine (HP:0032944): observed,17
1,Alkaline urine (HP:0032944): not measured,1


In [10]:
# maximumOsm(urine) >  940
conc = HpTerm(hpo_id="HP:0004727", label="Impaired renal concentrating ability")
concT = Thresholder(unit="mOsm/kg", threshold_low=940, hpo_term_low=conc)
concTM = ThresholdedColumnMapper(column_name="maximumOsm(urine)", thresholder=concT)
column_mapper_list.append(concTM)
concTM.preview_column(df)

,mapping: 940.0-None mOsm/kg,count
0,Impaired renal concentrating ability (HP:0004727): observed,13
1,Impaired renal concentrating ability (HP:0004727): not measured,5


In [11]:
#  HP:0008341 Distal renal tubular acidosis
dRTA = SimpleColumnMapper(column_name="dRTA", hpo_id="HP:0008341", hpo_label="Distal renal tubular acidosis", observed="Yes", excluded="No")
column_mapper_list.append(dRTA)
dRTA.preview_column(df)

,mapping,count
0,"original value: ""Yes"" -> HP: Distal renal tubular acidosis (HP:0008341) (observed)",17
1,"original value: ""No"" -> HP: Distal renal tubular acidosis (HP:0008341) (excluded)",1


In [12]:
df["Sex"] =  df["Sex/Age"].apply(lambda x: x.split("/")[0])
df["Age"] =  df["Sex/Age"].apply(lambda x: x.split("/")[1])
ageMapper = AgeColumnMapper.by_year(column_name="Age")
sexMapper = SexColumnMapper(male_symbol="M", female_symbol="F", column_name="Sex")

In [13]:
SLC4A1_transcript = "NM_000342.4"
vvalidator = VariantValidator(genome_build="hg38", transcript=SLC4A1_transcript)
var_d = {}
for v in df["NM_000342.4"].unique():
    var = vvalidator.encode_hgvs(v)
    var_d[v] = var
varMapper = VariantColumnMapper(variant_column_name="NM_000342.4", variant_d=var_d, default_genotype="heterozygous")

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000342.4%3Ac.1766G>A/NM_000342.4?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000342.4%3Ac.1765C>T/NM_000342.4?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_000342.4%3Ac.1838C>T/NM_000342.4?content-type=application%2Fjson


In [14]:
encoder = CohortEncoder(df=df, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list,
                        individual_column_name="individual", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata)
 	 
rta1 = Disease(disease_id='OMIM:179800', disease_label='Distal renal tubular acidosis 1')
encoder.set_disease(rta1)

In [15]:
individuals = encoder.get_individuals()

In [16]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
INFORMATION,NOT_MEASURED,6


In [17]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
A:I:1 (FEMALE; P60Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrolithiasis (HP:0000787); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Nephrocalcinosis (HP:0000121); excluded: Abnormal circulating creatinine concentration (HP:0012100); excluded: Decreased serum bicarbonate concentration (HP:0032066); excluded: Abnormal blood potassium concentration (HP:0011042)
A:I:2 (FEMALE; P40Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Nephrolithiasis (HP:0000787); Decreased serum bicarbonate concentration (HP:0032066); Hypokalemia (HP:0002900); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); excluded: Abnormal circulating creatinine concentration (HP:0012100); excluded: Distal renal tubular acidosis (HP:0008341)
A:I:3 (FEMALE; P44Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Elevated circulating creatinine concentration (HP:0003259); Decreased serum bicarbonate concentration (HP:0032066); Alkaline urine (HP:0032944); Distal renal tubular acidosis (HP:0008341); excluded: Nephrolithiasis (HP:0000787); excluded: Abnormal blood potassium concentration (HP:0011042)
A:II:3 (MALE; P27Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Nephrolithiasis (HP:0000787); Hypokalemia (HP:0002900); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Abnormal circulating creatinine concentration (HP:0012100); excluded: Decreased serum bicarbonate concentration (HP:0032066)
A:II:5 (FEMALE; P35Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Nephrocalcinosis (HP:0000121); excluded: Nephrolithiasis (HP:0000787); excluded: Abnormal circulating creatinine concentration (HP:0012100); excluded: Decreased serum bicarbonate concentration (HP:0032066); excluded: Abnormal blood potassium concentration (HP:0011042)
A:II:6 (MALE; P36Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Decreased serum bicarbonate concentration (HP:0032066); Hypokalemia (HP:0002900); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Nephrolithiasis (HP:0000787); excluded: Abnormal circulating creatinine concentration (HP:0012100)
A:II:8 (MALE; P34Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrocalcinosis (HP:0000121); Nephrolithiasis (HP:0000787); Elevated circulating creatinine concentration (HP:0003259); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Decreased serum bicarbonate concentration (HP:0032066); excluded: Abnormal blood potassium concentration (HP:0011042)
A:II:9 (MALE; P31Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Nephrolithiasis (HP:0000787); Alkaline urine (HP:0032944); Impaired renal concentrating ability (HP:0004727); Distal renal tubular acidosis (HP:0008341); excluded: Nephrocalcinosis (HP:0000121); excluded: Abnormal circulating creatinine concentration (HP:0012100); excluded: Decreased serum bicarbonate concentration (HP:0032066); excluded: Abnormal blood potassium concentration (HP:0011042)
A:III:1 (MALE; P9Y),Distal renal tubular acidosis 1 (OMIM:179800),NM_000342.4:c.1766G>A (heterozygous),Alkaline urine (HP:0032944); Distal renal tubular acidosis (HP:0008341); excluded: Nephrocalcinosi

In [18]:
Individual.output_individuals_as_phenopackets(individual_list=individuals, metadata=metadata)

We output 18 GA4GH phenopackets to the directory phenopackets
